# Rf 5 0 9_Wsinteractive
Organization and simultaneous fits: easy interactive access to workspace contents - CINT to CLING code migration



**Author:** 04/2009 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:56 AM.</small></i>

In [1]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

In [2]:
void fillWorkspace(RooWorkspace &w);

 A helper function is created: 

In [3]:
%%cpp -d
void fillWorkspace(RooWorkspace &w)
{
   // C r e a t e  p d f   a n d   f i l l   w o r k s p a c e
   // --------------------------------------------------------

   // Declare observable x
   RooRealVar x("x", "x", 0, 10);

   // Create two Gaussian PDFs g1(x,mean1,sigma) anf g2(x,mean2,sigma) and their parameters
   RooRealVar mean("mean", "mean of gaussians", 5, 0, 10);
   RooRealVar sigma1("sigma1", "width of gaussians", 0.5);
   RooRealVar sigma2("sigma2", "width of gaussians", 1);

   RooGaussian sig1("sig1", "Signal component 1", x, mean, sigma1);
   RooGaussian sig2("sig2", "Signal component 2", x, mean, sigma2);

   // Build Chebychev polynomial p.d.f.
   RooRealVar a0("a0", "a0", 0.5, 0., 1.);
   RooRealVar a1("a1", "a1", 0.2, 0., 1.);
   RooChebychev bkg("bkg", "Background", x, RooArgSet(a0, a1));

   // Sum the signal components into a composite signal p.d.f.
   RooRealVar sig1frac("sig1frac", "fraction of component 1 in signal", 0.8, 0., 1.);
   RooAddPdf sig("sig", "Signal", RooArgList(sig1, sig2), sig1frac);

   // Sum the composite signal and background
   RooRealVar bkgfrac("bkgfrac", "fraction of background", 0.5, 0., 1.);
   RooAddPdf model("model", "g1+g2+a", RooArgList(bkg, sig), bkgfrac);

   w.import(model);
}

Create and fill workspace
 ------------------------------------------------

Create a workspace named 'w'
 With CINT w could exports its contents to
 a same-name C++ namespace in CINT 'namespace w'.
 but this does not work anymore in CLING.
 so this tutorial is an example on how to
 change the code

In [4]:
RooWorkspace *w1 = new RooWorkspace("w", kTRUE);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

[#1] INFO:ObjectHandling -- RooWorkspace::exportToCint(w) INFO: references to all objects in this workspace will be created in CINT in 'namespace w'


Fill workspace with p.d.f. and data in a separate function

In [5]:
fillWorkspace(*w1);

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooAddPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooChebychev::bkg
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::x
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::a0
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::a1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::bkgfrac
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooAddPdf::sig
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooGaussian::sig1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::mean
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::sigma1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::sig1frac
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooGaussian::sig2
[#1] INFO:ObjectHandling -- RooWorkspace::import(

Print workspace contents

In [6]:
w1->Print();


RooWorkspace(w) w contents

variables
---------
(a0,a1,bkgfrac,mean,sig1frac,sigma1,sigma2,x)

p.d.f.s
-------
RooChebychev::bkg[ x=x coefList=(a0,a1) ] = 0.8
RooAddPdf::model[ bkgfrac * bkg + [%] * sig ] = 0.9
RooAddPdf::sig[ sig1frac * sig1 + [%] * sig2 ] = 1
RooGaussian::sig1[ x=x mean=mean sigma=sigma1 ] = 1
RooGaussian::sig2[ x=x mean=mean sigma=sigma2 ] = 1



This does not work anymore with cling
 use normal workspace functionality

Use workspace contents
 ----------------------------------------------

Old syntax to use the name space prefix operator to access the workspace contents

 RooDataSet* d = w::model.generate(w::x,1000) ;
 RooFitResult* r = w::model.fitTo(*d) ;

Use normal workspace methods

In [7]:
RooAbsPdf *model = w1->pdf("model");
RooRealVar *x = w1->var("x");

RooDataSet *d = model->generate(*x, 1000);
RooFitResult *r = model->fitTo(*d);

[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_model_FOR_OBS_x with 0 entries
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (bkg,sig1,sig2)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a0           5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 a1           2.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     3 bkgfrac      5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     4 mean         5.00000e+00  1.00000e+00    0.00000e+00  1.00000e+01
     5 sig1frac     8.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR   

Old syntax to access the variable x
 RooPlot* frame = w::x.frame() ;

In [8]:
RooPlot *frame = x->frame();
d->plotOn(frame);

Old syntax to omit x::
 NB: The 'w::' prefix can be omitted if namespace w is imported in local namespace
 in the usual C++ way

 using namespace w;
 model.plotOn(frame) ;
 model.plotOn(frame,Components(bkg),LineStyle(kDashed)) ;

New correct syntax

In [9]:
RooAbsPdf *bkg = w1->pdf("bkg");
model->plotOn(frame);
model->plotOn(frame, Components(*bkg), LineStyle(kDashed));

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (bkg)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()


Draw the frame on the canvas

In [10]:
new TCanvas("rf509_wsinteractive", "rf509_wsinteractive", 600, 600);
gPad->SetLeftMargin(0.15);
frame->GetYaxis()->SetTitleOffset(1.4);
frame->Draw();

Draw all canvases 

In [11]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()